<a href="https://colab.research.google.com/github/geonhong/mlcfd/blob/master/mlcfd_ex01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Regression ##

## Import Data ##

In [2]:
import requests
import shutil

import numpy as np

#datin = np.load('gdrive/My Drive/Works/mlcfd/volfrac_data.npy')
#target = np.load('gdrive/My Drive/Works/mlcfd/volfrac_target.npy')

def load_from_url(url):
  resp = requests.get(url, stream=True)
  
  with open('tmp.npy', 'wb') as f:
    shutil.copyfileobj(resp.raw, f)
   
  var = np.load('tmp.npy')
  
  return var

datin = load_from_url('https://github.com/geonhong/mlcfd/blob/master/volfrac/samples/volfrac_data.npy?raw=true')
target = load_from_url('https://github.com/geonhong/mlcfd/blob/master/volfrac/samples/volfrac_target.npy?raw=true')

print(datin.shape)
print(target.shape)

(122, 64)
(122,)


## Build model ##

In [3]:
from keras import models
from keras import layers

def build_model():
  model = models.Sequential()
  model.add(layers.Dense(16, activation='relu', input_shape=(datin.shape[1],)))
  model.add(layers.Dense(16, activation='relu', input_shape=(datin.shape[1],)))
  model.add(layers.Dense(1))
  model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
  return model



model = build_model()
# model.fit(datin, target, epochs=num_epochs, batch_size=1)

Using TensorFlow backend.
W0620 10:19:20.054272 139691811927936 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0620 10:19:20.096915 139691811927936 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0620 10:19:20.108005 139691811927936 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0620 10:19:20.150928 139691811927936 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



Load a diamond volume fraction and predict the result

### Manipulate data input

In [4]:
dataset = datin
targets = target

print(dataset.shape)
print(targets.shape)

# Shuffle data
index = np.arange(len(dataset))
np.random.shuffle(index)

train_data = []
train_targ = []

test_data = []
test_targ = []

i = 0
for itrg in index:
  if i<100:
    train_data.append(dataset[itrg])
    train_targ.append(targets[itrg])
  else:
    test_data.append(dataset[itrg])
    test_targ.append(targets[itrg])

  i += 1

train_data = np.array(train_data)
train_targ = np.array(train_targ)

test_data = np.array(test_data)
test_targ = np.array(test_targ)
    
print("train data shape: ", train_data.shape)
print("train target shape: ", train_targ.shape)
print("test data shape: ", test_data.shape)
print("test target shape: ", test_targ.shape)

(122, 64)
(122,)
train data shape:  (100, 64)
train target shape:  (100,)
test data shape:  (22, 64)
test target shape:  (22,)


### Build a model and evaluate the results



In [13]:
num_epochs = 20

model.fit(train_data, train_targ, epochs=num_epochs, batch_size=1)

val_mse, val_mae = model.evaluate(test_data, test_targ)

print("MSE: ", val_mse)
print("MAE: ", val_mae)



Epoch 1/20
100/100 [==============================] - 0s 3ms/step - loss: 0.6260 - mean_absolute_error: 0.6299
Epoch 2/20
100/100 [==============================] - 0s 1ms/step - loss: 0.1551 - mean_absolute_error: 0.3187
Epoch 3/20
100/100 [==============================] - 0s 1ms/step - loss: 0.0707 - mean_absolute_error: 0.2045
Epoch 4/20
100/100 [==============================] - 0s 1ms/step - loss: 0.0649 - mean_absolute_error: 0.1944
Epoch 5/20
100/100 [==============================] - 0s 1ms/step - loss: 0.0537 - mean_absolute_error: 0.1692
Epoch 6/20
100/100 [==============================] - 0s 1ms/step - loss: 0.0424 - mean_absolute_error: 0.1613
Epoch 7/20
100/100 [==============================] - 0s 1ms/step - loss: 0.0356 - mean_absolute_error: 0.1412
Epoch 8/20
100/100 [==============================] - 0s 1ms/step - loss: 0.0309 - mean_absolute_error: 0.1217
Epoch 9/20
100/100 [==============================] - 0s 1ms/step - loss: 0.0265 - mean_absolute_error: 0.1114
E

In [14]:
i = 0

cd_pred = model.predict(test_data)

for i in range(0, len(cd_pred)):
  
  diff = float(test_targ[i]) - float(cd_pred[i])
  err = diff/float(test_targ[i])
  
  print(i, test_targ[i], float(cd_pred[i]), diff, err)

0 1.9363976264032607 1.9681545495986938 -0.03175692319543311 -0.016400001096066014
1 0.9152300797205262 0.9999894499778748 -0.08475937025734859 -0.09260990447694926
2 1.8709652016968163 1.810355305671692 0.060609896025124455 0.03239498841034355
3 2.18026754497884 2.407454013824463 -0.22718646884562288 -0.10420118822978112
4 2.5750032470757622 2.7005088329315186 -0.12550558585575633 -0.04873997188092232
5 2.2499093918537754 2.8095333576202393 -0.5596239657664639 -0.24873177906305421
6 1.8765877319897402 2.0136735439300537 -0.13708581194031355 -0.07305057451002404
7 0.6141546786888715 0.7912402153015137 -0.17708553661264215 -0.2883402874837546
8 2.4979940772720965 2.668829917907715 -0.1708358406356183 -0.0683892096422332
9 2.6297066112219447 2.745307683944702 -0.11560107272275744 -0.04395968441096976
10 0.5768186547945997 0.7597330808639526 -0.18291442606935293 -0.3171090680735477
11 0.2126918943935795 0.30715614557266235 -0.09446425117908286 -0.4441365828651646
12 2.1133330774668684 2.5